# 03 - Inference Pipeline

This notebook demonstrates the end-to-end facial keypoint detection pipeline using the modernized `facial_keypoints` package.

## Overview

- Use `FacialKeypointsPipeline` for complete face detection + keypoint prediction
- Process images with multiple faces
- Visualize results with bounding boxes and keypoints
- Explore individual pipeline components

In [ ]:
# Standard imports
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Package imports
from facial_keypoints.pipeline import FacialKeypointsPipeline, PipelineResult
from facial_keypoints.detection.face_detector import FaceDetector, BoundingBox
from facial_keypoints.models.predictor import KeypointPredictor
from facial_keypoints.visualization.plotting import (
    plot_keypoints,
    plot_face_detections,
    plot_pipeline_result,
)

# Display settings
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 10)

## 1. Initialize the Pipeline

The `FacialKeypointsPipeline` combines face detection (Haar Cascades) with keypoint prediction (CNN).

In [ ]:
# Initialize the pipeline
# Note: Requires cascade file and trained model
try:
    pipeline = FacialKeypointsPipeline(
        cascade_path="detector_architectures/haarcascade_frontalface_default.xml",
        model_path="models/keypoint_model.keras",  # Or your trained model path
    )
    print("✓ Pipeline initialized successfully!")
    print(f"  Cascade: {pipeline.face_detector.cascade_path}")
    print(f"  Model: {pipeline.keypoint_predictor.model_path}")
except Exception as e:
    print(f"Could not initialize pipeline: {e}")
    print("\nMake sure you have:")
    print("  1. Haar cascade file in detector_architectures/")
    print("  2. Trained model in models/ (run notebook 02 first)")

## 2. Process a Single Image

Use `pipeline.process()` to detect faces and predict keypoints in one call.

In [ ]:
# Process an image
try:
    # Load a test image (update path as needed)
    image_path = "images/james.jpg"  # Example image from the project
    
    if not Path(image_path).exists():
        print(f"Image not found: {image_path}")
        print("Update the path to an existing image file.")
    else:
        result = pipeline.process(image_path)
        
        print(f"Processed image: {image_path}")
        print(f"Image shape: {result.image.shape}")
        print(f"Faces detected: {result.n_faces}")
        
        for i, face in enumerate(result.faces):
            box = face.bounding_box
            print(f"\nFace {i + 1}:")
            print(f"  Bounding box: ({box.x}, {box.y}, {box.width}×{box.height})")
            print(f"  Center: {box.center}")
            print(f"  Keypoints shape: {face.keypoints.shape}")
except NameError:
    print("Pipeline not initialized - run the cell above first")

## 3. Visualize Results

In [ ]:
# Visualize the pipeline results
try:
    fig = plot_pipeline_result(
        result.image,
        result.faces,
        figsize=(12, 12),
        show_boxes=True,
        show_keypoints=True,
        keypoint_color='cyan',
        keypoint_size=50,
    )
    plt.show()
except NameError:
    print("No results to visualize - run the processing cell first")

## 4. Process Multiple Images

Batch process multiple images from a directory.

In [ ]:
def process_directory(pipeline, image_dir, extensions=('.jpg', '.jpeg', '.png')):
    """Process all images in a directory."""
    image_dir = Path(image_dir)
    results = []
    
    for ext in extensions:
        for image_path in image_dir.glob(f'*{ext}'):
            try:
                result = pipeline.process(str(image_path))
                results.append((image_path.name, result))
                print(f"✓ {image_path.name}: {result.n_faces} face(s)")
            except Exception as e:
                print(f"✗ {image_path.name}: {e}")
    
    return results

# Process all images in the images/ directory
try:
    if Path('images').exists():
        all_results = process_directory(pipeline, 'images')
        print(f"\nProcessed {len(all_results)} images")
    else:
        print("images/ directory not found")
except NameError:
    print("Pipeline not initialized")

## 5. Compare Detection Methods

Show different ways to process faces: detect all vs. detect single largest.

In [ ]:
# Compare detect_all=True vs detect_all=False
try:
    image_path = "images/obamas.jpg"  # An image with multiple faces
    
    if Path(image_path).exists():
        fig, axes = plt.subplots(1, 2, figsize=(16, 8))
        
        # Detect all faces
        result_all = pipeline.process(image_path, detect_all=True)
        img_rgb = cv2.cvtColor(result_all.image.copy(), cv2.COLOR_BGR2RGB)
        
        # Draw all faces
        for face in result_all.faces:
            box = face.bounding_box
            cv2.rectangle(img_rgb, (box.x, box.y), 
                          (box.x + box.width, box.y + box.height), (0, 255, 0), 3)
        axes[0].imshow(img_rgb)
        axes[0].set_title(f'detect_all=True ({result_all.n_faces} faces)')
        axes[0].axis('off')
        
        # Detect single (largest) face
        result_single = pipeline.process(image_path, detect_all=False)
        img_rgb2 = cv2.cvtColor(result_single.image.copy(), cv2.COLOR_BGR2RGB)
        
        for face in result_single.faces:
            box = face.bounding_box
            cv2.rectangle(img_rgb2, (box.x, box.y),
                          (box.x + box.width, box.y + box.height), (255, 0, 0), 3)
        axes[1].imshow(img_rgb2)
        axes[1].set_title(f'detect_all=False (largest face only)')
        axes[1].axis('off')
        
        plt.tight_layout()
        plt.show()
    else:
        print(f"Multi-face image not found: {image_path}")
except NameError:
    print("Pipeline not initialized")

## 6. Using Individual Components

You can also use `FaceDetector` and `KeypointPredictor` separately for more control.

In [ ]:
# Use FaceDetector independently
try:
    detector = FaceDetector(
        cascade_path="detector_architectures/haarcascade_frontalface_default.xml",
        scale_factor=1.2,
        min_neighbors=5,
    )
    
    image = cv2.imread("images/james.jpg")
    if image is not None:
        boxes = detector.detect(image)
        
        print(f"Detected {len(boxes)} faces:")
        for i, box in enumerate(boxes):
            print(f"  Face {i + 1}: x={box.x}, y={box.y}, "
                  f"size={box.width}×{box.height}, area={box.area}")
        
        # Visualize
        fig = plot_face_detections(image, boxes, title="Face Detection Results")
        plt.show()
    else:
        print("Could not load image")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Use KeypointPredictor independently
try:
    predictor = KeypointPredictor(
        model_path="models/keypoint_model.keras",
        image_size=96,
    )
    
    # Get a face crop
    if len(boxes) > 0:
        box = boxes[0]
        face_crop = detector.crop_face(image, box, padding=0.1)
        
        # Predict keypoints
        prediction = predictor.predict(face_crop, denormalize=True)
        
        print(f"Raw output shape: {prediction.raw_output.shape}")
        print(f"Keypoints shape: {prediction.keypoints.shape}")
        print(f"\nKeypoint coordinates (in 96×96 space):")
        for i, (x, y) in enumerate(prediction.keypoints):
            print(f"  Keypoint {i:2d}: ({x:6.2f}, {y:6.2f})")
        
        # Visualize on crop
        fig, ax = plt.subplots(figsize=(6, 6))
        gray_crop = cv2.cvtColor(face_crop, cv2.COLOR_BGR2GRAY)
        gray_resized = cv2.resize(gray_crop, (96, 96))
        plot_keypoints(gray_resized, prediction.keypoints.flatten(), ax=ax)
        ax.set_title('Keypoints on Face Crop')
        plt.show()
except NameError:
    print("Detector or boxes not available")
except Exception as e:
    print(f"Error: {e}")

## 7. Process from Webcam (Optional)

Real-time keypoint detection using webcam input.

In [ ]:
def capture_and_process(pipeline, n_frames=1):
    """Capture frames from webcam and process them."""
    cap = cv2.VideoCapture(0)
    results = []
    
    try:
        for i in range(n_frames):
            ret, frame = cap.read()
            if not ret:
                print("Could not capture frame")
                break
            
            result = pipeline.process(frame)
            results.append(result)
            print(f"Frame {i + 1}: {result.n_faces} face(s) detected")
    finally:
        cap.release()
    
    return results

# Uncomment to capture from webcam
# try:
#     webcam_results = capture_and_process(pipeline, n_frames=1)
#     if webcam_results:
#         fig = plot_pipeline_result(webcam_results[0].image, webcam_results[0].faces)
#         plt.show()
# except Exception as e:
#     print(f"Webcam error: {e}")

print("Webcam capture disabled. Uncomment the code above to enable.")

## 8. Export Results

Save processed images or keypoint coordinates.

In [ ]:
def save_result(result, output_path, draw_boxes=True, draw_keypoints=True):
    """Save processed result to an image file."""
    img = result.image.copy()
    
    for face in result.faces:
        box = face.bounding_box
        
        if draw_boxes:
            cv2.rectangle(img, (box.x, box.y),
                          (box.x + box.width, box.y + box.height), (0, 255, 0), 2)
        
        if draw_keypoints:
            for x, y in face.keypoints:
                cv2.circle(img, (int(x), int(y)), 3, (0, 255, 255), -1)
    
    cv2.imwrite(str(output_path), img)
    print(f"Saved: {output_path}")

# Save a processed result
try:
    output_dir = Path('output')
    output_dir.mkdir(exist_ok=True)
    
    save_result(result, output_dir / 'processed_result.jpg')
except NameError:
    print("No result to save")

In [ ]:
# Export keypoints to JSON
import json

def export_keypoints_json(result, output_path):
    """Export keypoints to JSON format."""
    data = {
        "n_faces": result.n_faces,
        "faces": []
    }
    
    for i, face in enumerate(result.faces):
        face_data = {
            "face_id": i,
            "bounding_box": {
                "x": int(face.bounding_box.x),
                "y": int(face.bounding_box.y),
                "width": int(face.bounding_box.width),
                "height": int(face.bounding_box.height),
            },
            "keypoints": [
                {"x": float(x), "y": float(y)}
                for x, y in face.keypoints
            ]
        }
        data["faces"].append(face_data)
    
    with open(output_path, 'w') as f:
        json.dump(data, f, indent=2)
    
    print(f"Exported keypoints to: {output_path}")

try:
    export_keypoints_json(result, output_dir / 'keypoints.json')
except NameError:
    print("No result to export")

## Summary

This notebook demonstrated:

1. **Pipeline Usage**: `FacialKeypointsPipeline` for end-to-end processing
2. **Visualization**: `plot_pipeline_result()` and related functions
3. **Component Access**: Using `FaceDetector` and `KeypointPredictor` independently
4. **Batch Processing**: Processing multiple images from a directory
5. **Export**: Saving results as images or JSON

### API Reference

```python
# High-level pipeline
from facial_keypoints.pipeline import FacialKeypointsPipeline
pipeline = FacialKeypointsPipeline(cascade_path, model_path)
result = pipeline.process(image)  # Returns PipelineResult

# Low-level components
from facial_keypoints.detection.face_detector import FaceDetector
from facial_keypoints.models.predictor import KeypointPredictor

detector = FaceDetector(cascade_path)
boxes = detector.detect(image)  # Returns list[BoundingBox]

predictor = KeypointPredictor(model_path)
prediction = predictor.predict(face_crop)  # Returns KeypointPrediction
```